In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 20)

In [2]:
df_train = pd.read_csv("../dataset/original/train.csv", escapechar="\\")
df_train.phone = df_train.phone.fillna(-1).astype(np.int64)

In [3]:
df_train.head()

,record_id,name,type,address,phone,email,modification,linked_id
0,10111147-T1,ArDIA ENTERPRISESmLTD.,entity,NaN,4102971003602,imquires@zoho.vo,typo,10111147
1,12172071-T0,ANDRÉ ALMEIDA BLANCO,officer,la Rua Pelotas; 209; apartamento 72; Bloco A; ...,-1,NaN,typo,12172071
2,10104491-T1,MOONTA HOLDINGS LTD.,entity,NaN,65013464145,NaN,typo,10104491
3,10068808,SPRINGFIELD INVESTMENTS GROUP LIMITED,entity,NaN,-1,consulting@zoho.de,original,10068808
4,13011715,GRIGORY UVAROV,officer,127273; MOSCOW; OTRADNAJA STREET; HOUSE 2; NO....,-1,NaN,original,13011715


In [4]:
import string
import unidecode
def create_email_letters_matrix(df):
    df = df[['record_id','email']]
    df.email =  df.email.fillna('')
    df.email = df.email.astype(str) # convert to string
    df.email = df.email.str.lower() # lowercase
    no_accents = []
    for s in df.email:
        no_accents.append(unidecode.unidecode(s))
    df.email = no_accents
    # create return matrix
    columns = ['record_id','email','a','b','c','d','e','f','g','h','i','j','k','l',
               'm','n','o','p','q','r','s','t','u','v','w','x','y','z','0','1','2',
              '3','4','5','6','7','8','9']
    email_letters_matrix = pd.DataFrame(columns=columns)
    email_letters_matrix.record_id = df.record_id.copy()
    email_letters_matrix.email = df.email.copy()
    # count occurence of each letter and add the columns to the return df
    for l in tqdm(['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
                   'o','p','q','r','s','t','u','v','w','x','y','z','0','1','2',
                   '3','4','5','6','7','8','9']):
        new_col = []
        for (i,n) in zip(email_letters_matrix.index, email_letters_matrix.email):
            new_col.append(n.count(l))
        email_letters_matrix[l] = new_col
    return email_letters_matrix

In [11]:
m_train = create_email_letters_matrix(df_train)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
df_test = pd.read_csv('../dataset/original/test.csv', escapechar='\\')
df_test.head()

,record_id,name,type,address,phone,email,modification
0,10051937-TST-MR,CHEYHAM CORPORATION LIMITED,entity,NaN,1.986537e+10,sales@protonmail.com,move unique
1,10070762-TST-MR,MEABIZ LIMITED,entity,NaN,3.374033e+11,NaN,move unique
2,10000304-TST-MR,CHAMPLE INVESTMENTS LTD.,entity,NaN,4.142094e+12,NaN,move unique
3,10151607-TST-MR,Concept I Media and Technology Company Limited,entity,NaN,1.480354e+10,NaN,move unique
4,10124701-TST-MR,ARRA RESOURCES LIMITED,entity,NaN,4.207630e+11,NaN,move unique


In [12]:
m_test = create_email_letters_matrix(df_test)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [14]:
from scipy import *
from scipy.sparse import *
import numpy as np
m_train_csr = csr_matrix(m_train.drop(['record_id','email'], axis=1))
m_test_csr = csr_matrix(m_test.drop(['record_id','email'], axis=1))

In [15]:
import similaripy as sim
#output = sim.cosine(m_test_csr, m_train_csr.T, k=10)

Done: 100%|██████████| 266955/266955 [16:17<00:00, 273.04it/s]            


In [18]:
save_npz('similarity_email_notebook.npz', output.tocsr())

In [19]:
out_csr = load_npz('similarity_email_notebook.npz')

In [20]:
out_csr[0].argmax()

637395

In [23]:
m_test.loc[0,'email']

'sales@protonmail.com'

In [25]:
m_train.loc[637395,'email']

'sales@protonmail.com'

In [30]:
df_train_email = df_train[~df_train.email.isna()]
df_train_email

,record_id,name,type,address,phone,email,modification,linked_id
0,10111147-T1,ArDIA ENTERPRISESmLTD.,entity,NaN,4102971003602,imquires@zoho.vo,typo,10111147
3,10068808,SPRINGFIELD INVESTMENTS GROUP LIMITED,entity,NaN,-1,consulting@zoho.de,original,10068808
5,10105687,ODIN GROUP INVESTMENTS S.A.,entity,NaN,49668906658,info@outlook.cz,original,10105687
6,10016805-T2,CAPRI SOLUTIONS LIMITED,entity,NaN,-1,help@CAPRI.SOLUTIONS.LxMITED.de,typo,10016805
7,12133421,Crossmatch S.A.,officer,"Akara building 24 De Castro Street, Wickham Ca...",-1,CrossmatchSA@aol.gov,original,12133421
8,12126772,LAURENCE POUNTNEY LTD,officer,Akara Building 24 De Castro St; Wickham Cay I;...,-1,support@LAURENCE.POUNTNEY.LTD.gov,original,12126772
11,10002433-NV0,COMMTECH FOUNDATION INC.,entity,1441 Brickell Avenue 1200 Four Seasons Tower M...,-1,consulting@yahoo.it,new_value,10002433
12,10164485,BROTHERHOOD FOUNDATION..,entity,NaN,14680064152,BROTHERHOODFOUNDATION@protonmail.co.uk,original,10164485
14,12188364,LI GUOBANG,officer,ROOM 102; BUILDING 37; NO.18; TAIPINGJIAO 4 RO...,-1,GUOBANG@icloud.com,original,12188364
16,10117892-T4,LONGXI INVESTMENT LTD.,entity,NaN,332588033405,info@outlook.com,typo,10117892


In [31]:
group = df_train_email[['email', 'linked_id']].groupby('linked_id').apply(lambda x: list(x['email']))

In [32]:
group

linked_id
10000001    [inquires@TIANSHENG.INDUSTRY.AND.TRADING.CO.LT...
10000002                                 [consulting@zoho.cz]
10000005      [help@FORTUNEMAKER.INVESTMENTS.CORPORATION.com]
10000007               [inquires@KENT.DEVELOPMENT.LIMITED.ch]
10000009    [support@AMARANDAN.LTD.de, support@AMARANDAN.L...
10000011                     [inquires@HUGH.POWER.LIMITED.de]
10000012                             [support@protonmail.com]
10000018    [inqxires@CiEM.DiT.Corp.ch, inquires@CHEM.D-T....
10000022            [inquires@PRESTIGE.INTERNATIONAL.CORP.ch]
10000023    [help@yahoo.com, info@hotmail.gov, info@hotmai...
                                  ...                        
15001101                 [JaynetDesireeKabilaKyungu@gmail.ch]
15001601    [StavrosPapastavrou@icloud.ch, StavrosPapastnk...
15001701                       [AttanShansonga@protonmail.de]
15002201    [MohammadMustafa@icloud.ch, MohammadMustafa@ic...
15002701                             [MauricioMacri@zoho.gov

In [45]:
from collections import Counter
link_mc_email = {}
for (l, emails) in tqdm(zip(group.keys(), group)):
    link_mc_email[l] = Counter(emails).most_common(1)[0][0]
most_common_email = pd.DataFrame.from_dict(link_mc_email, orient='index', columns=['most_common_email'])
df_train_email_mc = pd.merge(df_train, most_common_email, how='left', left_on='linked_id', right_index=True)

In [46]:
df_train_email_mc

,record_id,name,type,address,phone,email,modification,linked_id,most_common_email
0,10111147-T1,ArDIA ENTERPRISESmLTD.,entity,NaN,4102971003602,imquires@zoho.vo,typo,10111147,inquires@zoho.it
1,12172071-T0,ANDRÉ ALMEIDA BLANCO,officer,la Rua Pelotas; 209; apartamento 72; Bloco A; ...,-1,NaN,typo,12172071,NaN
2,10104491-T1,MOONTA HOLDINGS LTD.,entity,NaN,65013464145,NaN,typo,10104491,NaN
3,10068808,SPRINGFIELD INVESTMENTS GROUP LIMITED,entity,NaN,-1,consulting@zoho.de,original,10068808,consulting@zoho.de
4,13011715,GRIGORY UVAROV,officer,127273; MOSCOW; OTRADNAJA STREET; HOUSE 2; NO....,-1,NaN,original,13011715,NaN
5,10105687,ODIN GROUP INVESTMENTS S.A.,entity,NaN,49668906658,info@outlook.cz,original,10105687,info@outlook.cz
6,10016805-T2,CAPRI SOLUTIONS LIMITED,entity,NaN,-1,help@CAPRI.SOLUTIONS.LxMITED.de,typo,10016805,help@CAPRI.SOLUTIONS.LIMITED.de
7,12133421,Crossmatch S.A.,officer,"Akara building 24 De Castro Street, Wickham Ca...",-1,CrossmatchSA@aol.gov,original,12133421,CrossmatchSA@aol.gov
8,12126772,LAURENCE POUNTNEY LTD,officer,Akara Building 24 De Castro St; Wickham Cay I;...,-1,support@LAURENCE.POUNTNEY.LTD.gov,original,12126772,support@LAURENCE.POUNTNEY.LTD.gov
9,11002523,"BREIT, MARCUS",intermediary,NaN,440180275310,NaN,original,11002523,NaN


In [47]:
df_train_email_mc = df_train_email_mc.drop_duplicates(subset=['linked_id','most_common_email']).drop(['record_id', 'email'], axis=1).reset_index(drop=True)
df_train_email_mc

,name,type,address,phone,modification,linked_id,most_common_email
0,ArDIA ENTERPRISESmLTD.,entity,NaN,4102971003602,typo,10111147,inquires@zoho.it
1,ANDRÉ ALMEIDA BLANCO,officer,la Rua Pelotas; 209; apartamento 72; Bloco A; ...,-1,typo,12172071,NaN
2,MOONTA HOLDINGS LTD.,entity,NaN,65013464145,typo,10104491,NaN
3,SPRINGFIELD INVESTMENTS GROUP LIMITED,entity,NaN,-1,original,10068808,consulting@zoho.de
4,GRIGORY UVAROV,officer,127273; MOSCOW; OTRADNAJA STREET; HOUSE 2; NO....,-1,original,13011715,NaN
5,ODIN GROUP INVESTMENTS S.A.,entity,NaN,49668906658,original,10105687,info@outlook.cz
6,CAPRI SOLUTIONS LIMITED,entity,NaN,-1,typo,10016805,help@CAPRI.SOLUTIONS.LIMITED.de
7,Crossmatch S.A.,officer,"Akara building 24 De Castro Street, Wickham Ca...",-1,original,12133421,CrossmatchSA@aol.gov
8,LAURENCE POUNTNEY LTD,officer,Akara Building 24 De Castro St; Wickham Cay I;...,-1,original,12126772,support@LAURENCE.POUNTNEY.LTD.gov
9,"BREIT, MARCUS",intermediary,NaN,440180275310,original,11002523,NaN


In [52]:
df_train_email_mc = df_train_email_mc.rename(columns={"linked_id":"record_id", "most_common_email":"email"})
df_train_email_mc = df_train_email_mc[~df_train_email_mc.email.isna()].reset_index()
df_train_email_mc

,index,name,type,address,phone,modification,record_id,email
0,0,ArDIA ENTERPRISESmLTD.,entity,NaN,4102971003602,typo,10111147,inquires@zoho.it
1,3,SPRINGFIELD INVESTMENTS GROUP LIMITED,entity,NaN,-1,original,10068808,consulting@zoho.de
2,5,ODIN GROUP INVESTMENTS S.A.,entity,NaN,49668906658,original,10105687,info@outlook.cz
3,6,CAPRI SOLUTIONS LIMITED,entity,NaN,-1,typo,10016805,help@CAPRI.SOLUTIONS.LIMITED.de
4,7,Crossmatch S.A.,officer,"Akara building 24 De Castro Street, Wickham Ca...",-1,original,12133421,CrossmatchSA@aol.gov
5,8,LAURENCE POUNTNEY LTD,officer,Akara Building 24 De Castro St; Wickham Cay I;...,-1,original,12126772,support@LAURENCE.POUNTNEY.LTD.gov
6,11,COMMTECH FOUNDATION INC.,entity,1441 Brickell Avenue 1200 Four Seasons Tower M...,-1,new_value,10002433,consulting@yahoo.it
7,12,BROTHERHOOD FOUNDATION..,entity,NaN,14680064152,original,10164485,BROTHERHOODFOUNDATION@protonmail.co.uk
8,14,LI GUOBANG,officer,ROOM 102; BUILDING 37; NO.18; TAIPINGJIAO 4 RO...,-1,original,12188364,GUOBANG@icloud.com
9,16,LONGXI INVESTMENT LTD.,entity,NaN,332588033405,typo,10117892,info@outlook.com


In [53]:
create_email_letters_matrix(df_train_email_mc)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,record_id,email,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,0,1,2,3,4,5,6,7,8,9
0,10111147,inquires@zoho.it,0,0,0,0,1,0,0,1,3,0,0,0,0,1,2,0,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,10068808,consulting@zoho.de,0,0,1,1,1,0,1,1,1,0,0,1,0,2,3,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,10105687,info@outlook.cz,0,0,1,0,0,1,0,0,1,0,1,1,0,1,4,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,10016805,help@capri.solutions.limited.de,1,0,1,2,3,0,0,1,4,0,0,3,1,1,2,2,0,1,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,12133421,crossmatchsa@aol.gov,3,0,2,0,0,0,1,1,0,0,0,1,1,0,3,0,0,1,3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,12126772,support@laurence.pountney.ltd.gov,1,0,1,1,3,0,1,0,0,0,0,2,0,3,3,3,0,2,1,3,3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,10002433,consulting@yahoo.it,1,0,1,0,0,0,1,1,2,0,0,1,0,2,3,0,0,0,1,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,10164485,brotherhoodfoundation@protonmail.co.uk,2,1,1,2,1,1,0,2,2,0,1,1,1,3,8,1,0,3,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,12188364,guobang@icloud.com,1,1,2,1,0,0,2,0,1,0,0,1,1,1,3,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,10117892,info@outlook.com,0,0,1,0,0,1,0,0,1,0,1,1,1,1,5,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
